In [1]:
pip install streamlit moviepy google-cloud-speech google-cloud-texttospeech openai


Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.cloud import speech

def transcribe_audio(file_path):
    client = speech.SpeechClient()
    with open(file_path, "rb") as audio_file:
        content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    response = client.recognize(config=config, audio=audio)
    transcript = "".join([result.alternatives[0].transcript for result in response.results])
    return transcript


In [4]:
import openai

openai.api_key = '22ec84421ec24230a3638d1b51e3a7dc'

def correct_transcription(transcript):
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=f"Correct this transcription:\n\n{transcript}",
        max_tokens=1024,
    )
    return response.choices[0].text.strip()


In [5]:
from google.cloud import texttospeech

def text_to_speech(corrected_text):
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=corrected_text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Wavenet-D",
    )
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.LINEAR16)
    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )
    with open("output_audio.wav", "wb") as out:
        out.write(response.audio_content)


In [6]:
from moviepy.editor import VideoFileClip, AudioFileClip

def replace_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)
    video = video.set_audio(audio)
    video.write_videofile("output_video.mp4", codec="libx264", audio_codec="aac")


In [13]:
import streamlit as st

st.title("Video Audio Replacement with AI")

video_file = st.file_uploader("Upload Video File",type=["mp4"])
if video_file is not None:
    with open("input_video.mp4", "wb") as f:
        f.write(video_file.getbuffer())
    st.video("input_video.mp4")
    
    if st.button("Process"):
        transcript = transcribe_audio("input_video.mp4")
        corrected_text = correct_transcription(transcript)
        text_to_speech(corrected_text)
        replace_audio("input_video.mp4", "output_audio.wav")
        st.video("output_video.mp4")


2024-10-28 14:19:54.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-28 14:19:54.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
